In [23]:
#import libraries
import numpy as np
import imutils
import time
from scipy import spatial
import cv2
import os

In [65]:
#parameters
FRAMES_BEFORE_CURRENT = 10
preDefinedConfidence = 0.5
preDefinedThreshold = 0.5
inputWidth, inputHeight = 416, 416 #Experimental

In [76]:
#loading yolo
LABELS = open('./yolo/coco.names').read().strip().split("\n")
yolo_weights = './yolo/yolov3.weights'
yolo_cfg = './yolo/yolov3.cfg'

In [67]:
# object want to count
list_of_vehicles = ["bicycle", "car", "motorbike", "bus", "truck", "train"]
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

In [68]:
#get frame and time
def countFPS(start_time, num_frames):
    current_time = int(time.time())
    if(current_time > start_time):
        os.system('clear')
        num_frames = 0
        start_time = current_time
    return start_time, num_frames

In [69]:
#display number of detected vehicles on frame
def displayCount(frame, vehicle_count):
    cv2.putText(frame,'Detected Vehicles: ' + str(vehicle_count),(20, 20),
                cv2.FONT_HERSHEY_SIMPLEX,0.8,(0, 255, 0),5,cv2.FONT_HERSHEY_COMPLEX_SMALL)

In [70]:
#draw rectangle on detected cars (chq if box exist->flatten the array->get corners for rec,draw rec and circle)
def drawDetectionBoxes(idxs, boxes, classIDs, confidences, frame):
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])
            color = [int(c) for c in COLORS[classIDs[i]]]
            cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            text = "{}: {:.4f}".format(LABELS[classIDs[i]],confidences[i])
            cv2.putText(frame, text, (x, y - 5),cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)
            cv2.circle(frame, (x + (w//2), y + (h//2)), 2, (0, 0xFF, 0), thickness=2)

In [71]:
#chq new detected box already in previous frames or not
def boxInPreviousFrames(previous_frame_detections, current_box, current_detections):
    centerX, centerY, width, height = current_box
    dist = np.inf  
    for i in range(FRAMES_BEFORE_CURRENT):
        coordinate_list = list(previous_frame_detections[i].keys())
        if len(coordinate_list) == 0:  
            continue
        temp_dist, index = spatial.KDTree(
            coordinate_list).query([(centerX, centerY)])
        if (temp_dist < dist):
            dist = temp_dist
            frame_num = i
            coord = coordinate_list[index[0]]

    if (dist > (max(width, height)/2)):
        return False
    current_detections[(centerX, centerY)] = previous_frame_detections[frame_num][coord]
    return True

In [72]:
#count the no of vehicles
def count_vehicles(idxs, boxes, classIDs, vehicle_count, previous_frame_detections, frame):
    current_detections = {}
    if len(idxs) > 0:
        for i in idxs.flatten():
            (x, y) = (boxes[i][0], boxes[i][1])
            (w, h) = (boxes[i][2], boxes[i][3])

            centerX = x + (w//2)
            centerY = y + (h//2)
            if (LABELS[classIDs[i]] in list_of_vehicles):
                current_detections[(centerX, centerY)] = vehicle_count
                if (not boxInPreviousFrames(previous_frame_detections, (centerX, centerY, w, h), current_detections)):
                    vehicle_count += 1
                ID = current_detections.get((centerX, centerY))
                if (list(current_detections.values()).count(ID) > 1):
                    current_detections[(centerX, centerY)] = vehicle_count
                    vehicle_count += 1
                cv2.putText(frame, str(ID), (centerX, centerY),cv2.FONT_HERSHEY_SIMPLEX, 0.5, [0, 0, 255], 2)
    return vehicle_count, current_detections

In [73]:
#load yolo from disk
net = cv2.dnn.readNetFromDarknet(yolo_cfg, yolo_weights)

ln = net.getLayerNames()
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]

In [74]:
previous_frame_detections = [{(0, 0): 0} for i in range(FRAMES_BEFORE_CURRENT)]
num_frames, vehicle_count = 0, 0
start_time = int(time.time())

In [75]:
cap = cv2.VideoCapture('test_data.mp4')
video_width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
video_height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

if cap.isOpened() == False:
    print('ERROR FILE NOT FOUND, OR WRONG CODEC USE!')

while cap.isOpened():
    ret,frame = cap.read();
    if ret == True:
        num_frames += 1
        boxes, confidences, classIDs = [], [], []
        start_time, num_frames = countFPS(start_time, num_frames)
        blob = cv2.dnn.blobFromImage(frame, 1 / 255.0, (inputWidth, inputHeight),swapRB=True, crop=False)
        net.setInput(blob)
        start = time.time()
        layerOutputs = net.forward(ln)
        end = time.time()
        for output in layerOutputs:
            for i, detection in enumerate(output):
                scores = detection[5:]
                classID = np.argmax(scores)
                confidence = scores[classID]
                if confidence > preDefinedConfidence:
                    box = detection[0:4] * np.array([video_width, video_height, video_width, video_height])
                    (centerX, centerY, width, height) = box.astype("int")
                    x = int(centerX - (width / 2))
                    y = int(centerY - (height / 2))
                    boxes.append([x, y, int(width), int(height)])
                    confidences.append(float(confidence))
                    classIDs.append(classID)

        idxs = cv2.dnn.NMSBoxes(boxes, confidences, preDefinedConfidence,preDefinedThreshold)
        drawDetectionBoxes(idxs, boxes, classIDs, confidences, frame)
        vehicle_count, current_detections = count_vehicles(idxs, boxes, classIDs, vehicle_count, previous_frame_detections, frame)
        displayCount(frame, vehicle_count)
        cv2.imshow('Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        previous_frame_detections.pop(0)
        previous_frame_detections.append(current_detections)
        
        if (cv2.waitKey(10) & 0xFF == ord('q')):
            break
    else:
        break


print("[INFO] cleaning up...")
cap.release()
cv2.destroyAllWindows()

[INFO] cleaning up...
